
# Analyzing replicability of functional connectivity-based multivariate BWAS on the Human Connectome Project dataset

This notebook contains the main analyses of the Matters Arising, with 6 example phenotypes.
For a comprehensive analysis of 52 HCP phenotypes, see `multivariate_BWAS_replicability_analysis_FC_extensive.ipynb`.
For an analysis with finer sample size resolution, see `multivariate_BWAS_replicability_analysis_FC_hires.ipynb`.
For an analysis with cortical thickness data, see `multivariate_BWAS_replicability_analysis_CT.ipynb`.

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from joblib import Parallel, delayed
from mlxtend.evaluate import permutation_test
sns.set(rc={"figure.figsize":(4, 2)})
sns.set_style("whitegrid")

## Load HCP data

We load functional network matrices (netmats) from the HCP1200-release, as published on connectomeDB: https://db.humanconnectome.org/
Due to licensing issues, data is not supplied with the repository, but can be downloaded from the ConnectomeDB or via `get_data.ipynb` (requires credentials).
See [readme.md](readme.md) for more details.

In [2]:
# HCP data can be obtained from the connectomeDB
# data is not part of this repository
subjectIDs = pd.read_csv('hcp_data/subjectIDs.txt', header=None)

netmats_pearson = pd.read_csv('hcp_data/netmats1_correlationZ.txt',
                             sep=' ',
                             header=None)
netmats_pearson['ID'] = subjectIDs[0]
netmats_pearson.set_index('ID', drop=True, inplace=True)


netmats_parcor = pd.read_csv('hcp_data/netmats2_partial-correlation.txt',
                             sep=' ',
                             header=None)
netmats_parcor['ID'] = subjectIDs[0]
netmats_parcor.set_index('ID', drop=True, inplace=True)

behavior = pd.read_csv('hcp_data/hcp1200_behavioral_data.csv')
behavior = behavior.set_index('Subject', drop=True)

# convert age to numeric
age = []
for s in behavior['Age']:
    if s == '36+':
        age.append(36)
    else:
        split = s.split(sep='-')
        age.append(np.mean((float(split[0]), float(split[1]))))

behavior['age'] = age
behavior.describe()

,T1_Count,T2_Count,3T_RS-fMRI_Count,3T_RS-fMRI_PctCompl,3T_tMRI_PctCompl,fMRI_WM_PctCompl,fMRI_Gamb_PctCompl,fMRI_Mot_PctCompl,fMRI_Lang_PctCompl,fMRI_Soc_PctCompl,...,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final,age
count,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,...,1204.000000,1204.000000,1201.000000,1205.000000,1200.000000,1200.000000,1198.000000,1195.000000,1195.000000,1206.000000
mean,1.478441,1.400498,3.509950,87.213267,88.219569,89.718076,89.852736,89.631675,87.027114,87.039801,...,110.421321,97.727500,1.449625,45.847718,95.166983,93.998533,1.845467,0.581590,1.822251,28.904229
std,0.635688,0.628216,1.215181,31.027886,29.942161,30.384864,30.163559,30.492092,33.566043,33.570248,...,9.107963,11.273251,1.783069,7.679288,14.583412,14.837851,0.541393,0.973172,0.542893,3.570475
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,82.740000,59.860000,0.000000,38.600000,56.350000,59.500000,1.560000,0.000000,1.080000,23.500000
25%,1.000000,1.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,101.120000,87.110000,0.000000,38.600000,84.070000,83.220000,1.800000,0.000000,1.760000,28.000000
50%,2.000000,1.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,108.790000,98.040000,1.000000,45.900000,95.360000,94.970000,1.800000,0.000000,1.800000,28.000000
75%,2.000000,2.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,122.250000,110.450000,2.000000,52.200000,105.570000,102.920000,1.880000,1.000000,1.840000,33.000000
max,2.000000,2.000000,4.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,122.250000,111.410000,10.000000,75.300000,134.650000,131.380000,15.000000,17.000000,15.000000,36.000000


# Helper function to prepare target variable


In [3]:
def create_data(target='CogTotalComp_AgeAdj', feature_data=netmats_parcor):
    # it's a good practice to use pandas for merging, messing up subject order can be painful
    features = feature_data.columns
    df = behavior
    df = df.merge(feature_data, left_index=True, right_index=True, how='left')

    df = df.dropna(subset = [target] + features.values.tolist())
    y = df[target].values
    X = df[features].values
    return X, y

# Helper function implementing a single bootstrap iteration

We define a workhorse function which:
- randomly samples the discovery and the replication datasets,
- creates cross-validated estimates of predictive performance within the discovery sample
- finalizes the model by fitting it to the whole discovery sample (overfits the discovery but not the replication sample)
- use it to predict the replication sample

In [4]:
def corr(X, Y):
    # just a small wrapper function (pandas correlation is silent in "unlucky" bootstraps with constant values)
    return pd.Series(X).corr( pd.Series(Y))

def bootstrap_workhorse(X, y, sample_size, model, random_state, shuffle_y=False):

    #create discovery and replication samples by random sampling from the whole dataset (without replacement)

    # if shuffle_y is true, a null model is created bz permuting y
    if shuffle_y:
        rng = np.random.default_rng(random_state)
        y = rng.permutation(y)

    # sample the discovery and replication sets *without replacement* (with replacement introduces spurious dependencies)
    X_discovery, X_replication, y_discovery, y_replication = train_test_split(X, y, train_size=sample_size, test_size=sample_size, shuffle=True, random_state=random_state)

    # standard 10-fold cross-validation
    cv = KFold(10)

    # below we obtain cross-validated predictions in the discovery sample
    predicted_discovery_cv = np.zeros_like(y_discovery)  # here we collect the predictions for each fold
    cor_per_fold = np.zeros(cv.n_splits)  # here we collect the predictive performance in each fold
    i = 0  # just a counter
    for train, test in cv.split(X=X_discovery, y=y_discovery):  # loop to leave one fold out
        model.fit(X=X_discovery[train], y=y_discovery[train]) # fit model to the training set
        predicted_discovery_cv[test] = model.predict(X=X_discovery[test]) # use fitted model to predict teh test set
        cor_per_fold[i] = corr(y_discovery[test], predicted_discovery_cv[test]) # calculate performance on tne test set
        i += 1
    # calculate mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # 'finalize' model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X_discovery, y=y_discovery)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X_discovery)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = corr(predicted_discovery_overfit, y_discovery)

    # We use the final model to predict the replication sample
    # This is correct (no overfitting here), the final model did not see this data during training
    predicted_replication = final_model.predict(X=X_replication)
    # we obtain the out-of-sample prediction performance estimates
    r_rep = corr(predicted_replication, y_replication)

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: corr(x, y),seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y_discovery, method='approximate', num_rounds=1000, func=lambda x, y: corr(x, y),seed=random_state)
    p_rep = permutation_test(predicted_replication, y_replication, method='approximate', num_rounds=1000, func=lambda x, y: corr(x, y),seed=random_state)
    # return results
    return r_disc_cv, r_disc_overfit, r_rep, p_disc_cv, p_disc_overfit, p_rep

All set, now we start the analysis.

# Replicability with sample sizes n=50, 100, 200, 300 and max
Here we train a few different models on 100 bootstrap samples.

We aggregate the results of our workhorse function in `n_bootstrap`=100 bootstrap cases (run in parallel).

The whole process is repeated for all sample sizes, fetaure_sets and target variables.

## Here we test age and 5 cognitive variables, including 'cognitive ability' (the main target variable in the target paper)
- age: age group of the participants
- CogTotalComp_AgeAdj: total cognitive ability
- PMAT24_A_CR, : Fluid Intelligence (Penn Progressive Matrices)
- CardSort_AgeAdj: Executive Function/Cognitive Flexibility (Dimensional Change Card Sort)
- Flanker_AgeAdj: Executive Function/Inhibition (Flanker Task)
- PicSeq_AgeAdj: Episodic Memory (Picture Sequence Memory)

# Reproducing the PCA+SVR-based model from the target paper
- Both PCA and SVR are done inside the cross-validation
- PCA reatains the firts k principal components that together explain 50% of the variance
- scikit-learn makes sure that PCA is only fit for the training samples
- both for the test sets (in the cross-validation) and the replication sample PCA is not re-fit, bt features are simply transformed with the already fit PCA

In [5]:
%%time
%%capture

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]);

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print('r discovery (with cv) :', tmp_data_frame.r_discovery_cv.mean(), 'r replication:', tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_PCA_SVR.csv')

CPU times: user 46.5 s, sys: 6.18 s, total: 52.7 s
Wall time: 3h 21min 54s


In [6]:
df.groupby(['connectivity', 'model', 'target', 'n']).mean()[['r_discovery_cv', 'r_replication']]

r_discovery_cv  r_replication
connectivity    model   target              n                                 
netmats_parcor  PCA_SVR CardSort_AgeAdj     50         0.021165       0.060743
                                            100        0.077595       0.082970
                                            200        0.097733       0.102561
                                            300        0.122905       0.123971
                                            500        0.139694       0.149550
                        CogTotalComp_AgeAdj 50         0.128129       0.140322
                                            100        0.190058       0.212629
                                            200        0.262374       0.271359
                                            300        0.303186       0.310136
                                            495        0.337404       0.346112
                        Flanker_AgeAdj      50         0.015871       0.028663
                                            100        0.038605       0.044466
                                            200        0.047259       0.058335
                                            300        0.073128       0.076713
                                            501        0.113786       0.122506
                        PMAT24_A_CR         50         0.097498       0.121062
                                            100        0.181649       0.166610
                                            200        0.224645       0.243301
                                            300        0.265809       0.283676
                                            499        0.312464       0.325274
                        PicSeq_AgeAdj       50         0.062696       0.075164
                                            100        0.063006       0.098759
                                            200        0.107779       0.124055
                                            300        0.142772       0.152545
                                            501        0.180642       0.185303
                        age                 50         0.184512       0.189014
                                            100        0.247298       0.273474
                                            200        0.342017       0.361751
                                            300        0.387432       0.394381
                                            501        0.426137       0.443717
netmats_pearson PCA_SVR CardSort_AgeAdj     50         0.013299       0.052865
                                            100        0.066240       0.055112
                                            200        0.075358       0.073260
                                            300        0.083765       0.090858
                                            500        0.105097       0.106389
                        CogTotalComp_AgeAdj 50         0.026131       0.072575
                                            100        0.042545       0.063498
                                            200        0.080139       0.106129
                                            300        0.123717       0.133728
                                            495        0.165746       0.175958
                        Flanker_AgeAdj      50         0.004188       0.026447
                                            100        0.017319       0.037797
                                            200        0.040002       0.047852
                                            300        0.066405       0.073938
                                            501        0.094710       0.102038
                        PMAT24_A_CR         50         0.011407       0.039374
                                            100        0.065420       0.052835
                                            200        0.083985       0.099913
                                            300        0.114977       0.141534
                    

# Now we fit a simple Ridge regression
(no feature selection, no hyperparameter optimization)
Based on some previous studies, this can be expected to perform better than SVR (especially with lower sample sizes). See the paper for details.

In [7]:
%%time
%%capture

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'ridge': Ridge()
}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set])

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })
                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print('r discovery (with cv) :', tmp_data_frame.r_discovery_cv.mean(), 'r replication:', tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_Ridge.csv')

CPU times: user 40.8 s, sys: 5.54 s, total: 46.3 s
Wall time: 27min 10s


In [8]:
df.groupby(['connectivity', 'model', 'target', 'n']).mean()[['r_discovery_cv', 'r_replication']]

r_discovery_cv  r_replication
connectivity    model target              n                                 
netmats_parcor  ridge CardSort_AgeAdj     50         0.033223       0.090787
                                          100        0.073555       0.101448
                                          200        0.119109       0.142017
                                          300        0.164847       0.167188
                                          500        0.178966       0.189577
                      CogTotalComp_AgeAdj 50         0.212834       0.256673
                                          100        0.308142       0.324413
                                          200        0.404524       0.406206
                                          300        0.431054       0.443890
                                          495        0.472652       0.479039
                      Flanker_AgeAdj      50         0.044371       0.069322
                                          100        0.073836       0.085089
                                          200        0.104340       0.104674
                                          300        0.117406       0.119512
                                          501        0.132261       0.140226
                      PMAT24_A_CR         50         0.182340       0.201273
                                          100        0.226739       0.227487
                                          200        0.272137       0.279880
                                          300        0.286111       0.290239
                                          499        0.292243       0.301492
                      PicSeq_AgeAdj       50         0.106290       0.098604
                                          100        0.094976       0.131433
                                          200        0.146384       0.170334
                                          300        0.170324       0.172413
                                          501        0.182331       0.185931
                      age                 50         0.242334       0.260920
                                          100        0.332321       0.342876
                                          200        0.395289       0.421317
                                          300        0.441473       0.448254
                                          501        0.480208       0.489671
netmats_pearson ridge CardSort_AgeAdj     50         0.052822       0.079214
                                          100        0.077086       0.113478
                                          200        0.114114       0.124521
                                          300        0.135054       0.146406
                                          500        0.143175       0.152942
                      CogTotalComp_AgeAdj 50         0.134584       0.215036
                                          100        0.246692       0.251511
                                          200        0.300071       0.315974
                                          300        0.335907       0.349330
                                          495        0.371412       0.381920
                      Flanker_AgeAdj      50         0.035977       0.057474
                                          100        0.073144       0.072340
                                          200        0.081574       0.088799
                                          300        0.096957       0.099712
                                          501        0.125087       0.119995
                      PMAT24_A_CR         50         0.149342       0.138880
                                          100        0.185662       0.183803
                                          200        0.201747       0.222919
                                          300        0.219045       0.232182
                                          499        0.232732       0.243257
                      PicSeq_AgeAdj       5

# Null scenario with random target
To evaluate false positives with biased estimates (Figure 1d).

First for PCA+SVR.

In [9]:
%%time
%%capture

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'PCA_SVR': Pipeline([('pca', PCA(n_components=0.5)),
                         ('svr', SVR())])

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]) # gives a random y when target is None

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel, with shuffle_y=True
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed, shuffle_y=True) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })

                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print('r discovery (with cv) :', tmp_data_frame.r_discovery_cv.mean(), 'r replication:', tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_null_PCA_SVR.csv')

df

CPU times: user 46.5 s, sys: 6 s, total: 52.5 s
Wall time: 3h 16min 32s


In [10]:
df.groupby(['connectivity', 'model', 'target', 'n']).mean()[['r_discovery_cv', 'r_replication']]

r_discovery_cv  r_replication
connectivity    model   target              n                                 
netmats_parcor  PCA_SVR CardSort_AgeAdj     50         0.028683       0.002786
                                            100        0.006824       0.011154
                                            200       -0.011921       0.004941
                                            300        0.000561      -0.003558
                                            500       -0.002693       0.004389
                        CogTotalComp_AgeAdj 50        -0.001601       0.004007
                                            100        0.005333      -0.012090
                                            200       -0.001798      -0.012032
                                            300        0.008406      -0.009813
                                            495       -0.007775      -0.002075
                        Flanker_AgeAdj      50         0.000604       0.007010
                                            100       -0.003850      -0.008456
                                            200       -0.005590       0.006889
                                            300       -0.006700       0.002094
                                            501       -0.001520       0.001972
                        PMAT24_A_CR         50         0.007739      -0.020072
                                            100        0.013193      -0.013371
                                            200        0.001066       0.000112
                                            300        0.017003      -0.009327
                                            499       -0.002256      -0.010670
                        PicSeq_AgeAdj       50        -0.024504       0.004879
                                            100        0.006898       0.001946
                                            200       -0.015076       0.002704
                                            300        0.012178      -0.005959
                                            501       -0.009452       0.002955
                        age                 50         0.015718      -0.009632
                                            100        0.021183       0.000831
                                            200        0.002010      -0.002066
                                            300        0.010896      -0.014503
                                            501        0.007978      -0.001730
netmats_pearson PCA_SVR CardSort_AgeAdj     50        -0.006625       0.005565
                                            100       -0.006670      -0.001255
                                            200       -0.007012      -0.001359
                                            300        0.014291      -0.000438
                                            500        0.000080       0.000920
                        CogTotalComp_AgeAdj 50         0.018770      -0.020559
                                            100        0.008058       0.000762
                                            200       -0.009799      -0.000800
                                            300       -0.000056       0.004032
                                            495       -0.004324       0.000941
                        Flanker_AgeAdj      50        -0.006951       0.013101
                                            100        0.002655      -0.011287
                                            200        0.003231       0.010932
                                            300       -0.009365       0.004417
                                            501       -0.008707       0.004297
                        PMAT24_A_CR         50         0.008029      -0.007791
                                            100        0.017448      -0.011428
                                            200        0.007621       0.007601
                                            300        0.013826       0.000286
                    

Then with Ridge.

In [11]:
%%time
%%capture

random_state = 42
n_bootstrap = 100

features = {
    'netmats_parcor': netmats_parcor,
    'netmats_pearson': netmats_pearson
}

models = {
    'Ridge': Ridge()

}

# We aggregate all results here:
df = pd.DataFrame(columns=['connectivity','model','target','n','r_discovery_cv','r_discovery_overfit','r_replication','p_discovery_cv','p_discovery_overfit','p_replication'])

for feature_set in features:
    for model in models:
        for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
            for sample_size in [50, 100, 200, 300, 'max']:

                print('*****************************************************************')
                print(feature_set, model, target_var, sample_size)

                X, y = create_data(target=target_var, feature_data=features[feature_set]) # gives a random y when target is None

                if sample_size=='max':
                    sample_size = int(len(y)/2)

                # create random seeds for each bootstrap iteration for reproducibility
                rng = np.random.default_rng(random_state)
                random_sates = rng.integers(np.iinfo(np.int32).max, size=n_bootstrap)

                # run bootstrap iterations in parallel, with shuffle_y=True
                r_discovery_cv, r_discovery_overfit, r_replication, p_discovery_cv, p_discovery_overfit, p_replication = zip(
                    *Parallel(n_jobs=-1)(
                    delayed(bootstrap_workhorse)(X, y, sample_size, models[model], seed, shuffle_y=True) for seed in random_sates))

                tmp_data_frame = pd.DataFrame({
                    'connectivity' : feature_set,
                    'model' : model,
                    'target' : target_var,
                    'n' : sample_size,
                    'r_discovery_cv': r_discovery_cv,
                    'r_discovery_overfit': r_discovery_overfit,
                    'r_replication': r_replication,
                    'p_discovery_cv': p_discovery_cv,
                    'p_discovery_overfit': p_discovery_overfit,
                    'p_replication': p_replication
                })

                #sns.scatterplot(x='r_replication', y='r_discovery_cv', data=tmp_data_frame)
                #plt.ylabel('in-sample (r)')
                #plt.xlabel('out-of-sample (r_pred)')
                #plt.show()
                print('r discovery (with cv) :', tmp_data_frame.r_discovery_cv.mean(), 'r replication:', tmp_data_frame.r_replication.mean())

                for alpha in [0.05, 0.01, 0.005, 0.001]:
                    print('Replicability at alpha =', alpha, ':',
                          (tmp_data_frame.loc[tmp_data_frame['p_discovery_cv']<alpha,'p_replication']<alpha).sum() / (tmp_data_frame['p_discovery_cv']<0.05).sum() * 100, '%')

                df = pd.concat((df, tmp_data_frame))
                df.reset_index(drop=True, inplace=True)
                df.to_csv('res/results_null_Ridge.csv')

df

CPU times: user 41.3 s, sys: 5.51 s, total: 46.8 s
Wall time: 26min 40s


In [12]:
df.groupby(['connectivity', 'model', 'target', 'n']).mean()[['r_discovery_cv', 'r_replication']]

r_discovery_cv  r_replication
connectivity    model target              n                                 
netmats_parcor  Ridge CardSort_AgeAdj     50         0.031644      -0.000121
                                          100        0.007479       0.014062
                                          200       -0.020488       0.010242
                                          300       -0.002089      -0.001656
                                          500       -0.000114      -0.001920
                      CogTotalComp_AgeAdj 50         0.022939      -0.003553
                                          100       -0.002252      -0.007803
                                          200       -0.017407       0.008146
                                          300       -0.009672      -0.005715
                                          495        0.008420      -0.001433
                      Flanker_AgeAdj      50        -0.005864       0.007939
                                          100        0.005748      -0.001361
                                          200        0.004708       0.004419
                                          300       -0.016153       0.006954
                                          501        0.009223       0.003327
                      PMAT24_A_CR         50         0.002180      -0.021616
                                          100       -0.003001      -0.012718
                                          200        0.000105       0.003351
                                          300       -0.012994      -0.005839
                                          499       -0.002584      -0.002607
                      PicSeq_AgeAdj       50        -0.027364      -0.012612
                                          100        0.021624      -0.001677
                                          200        0.011211       0.001994
                                          300        0.001884      -0.003351
                                          501        0.002581      -0.004661
                      age                 50        -0.014349      -0.019866
                                          100        0.011020       0.008189
                                          200        0.014167      -0.004365
                                          300       -0.002140      -0.003427
                                          501       -0.000934       0.007506
netmats_pearson Ridge CardSort_AgeAdj     50         0.032022      -0.035335
                                          100       -0.013893      -0.005578
                                          200       -0.003955      -0.001402
                                          300        0.000762      -0.002684
                                          500        0.001026       0.004724
                      CogTotalComp_AgeAdj 50         0.044520       0.004646
                                          100       -0.013345       0.001725
                                          200       -0.005666       0.010043
                                          300        0.004127      -0.000221
                                          495        0.002043      -0.005491
                      Flanker_AgeAdj      50        -0.007542       0.003721
                                          100        0.001752       0.005816
                                          200       -0.002066       0.013489
                                          300       -0.019641       0.006051
                                          501        0.003926      -0.002151
                      PMAT24_A_CR         50        -0.002403       0.003701
                                          100       -0.012464      -0.014601
                                          200        0.000571      -0.001337
                                          300        0.001551      -0.005956
                                          499        0.002485      -0.005751
                      PicSeq_AgeAdj       5

*See the notebook called 'plot_results.ipynb' for the results.*

## Now we train on the whole dataset, to obtain effect size estimates on N=1000.

With PCA+SVR:

In [13]:
model = Pipeline([('pca', PCA(n_components=0.5)), ('svr', SVR())])
random_state = 42
cv = KFold(10, shuffle=True, random_state=random_state)

bar_data_svr = []

for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
    print(target_var)
    X, y = create_data(target=target_var, feature_data=netmats_pearson)

    predicted_discovery_cv = np.zeros_like(y)
    cor_per_fold = np.zeros(cv.n_splits)
    i = 0
    for train, test in cv.split(X=X, y=y):
        model.fit(X=X[train], y=y[train])
        predicted_discovery_cv[test] = model.predict(X=X[test])
        cor_per_fold[i] = np.corrcoef(y[test], predicted_discovery_cv[test])[0,1]
        i += 1
    # correlation between the cross-validated predictions and observations in the discovery sample
    # this is the correct, unbiased estimate!
    # calculated as mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # finalize model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X, y=y)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)

    bar_data_svr.append(r_disc_cv)

    print('r =', np.round(r_disc_cv, 2), '\tp =', np.round(p_disc_cv, 3), '\tR2 =', np.round(r_disc_cv**2 * 100, 1),  '%')

age
r = 0.21 	p = 0.001 	R2 = 4.2 %
CogTotalComp_AgeAdj
r = 0.2 	p = 0.001 	R2 = 3.9 %
PMAT24_A_CR
r = 0.21 	p = 0.001 	R2 = 4.4 %
Flanker_AgeAdj
r = 0.12 	p = 0.001 	R2 = 1.5 %
CardSort_AgeAdj
r = 0.15 	p = 0.001 	R2 = 2.1 %
PicSeq_AgeAdj
r = 0.15 	p = 0.001 	R2 = 2.2 %


With Ridge and Pearson correlation:

In [14]:
model = Ridge()
random_state = 42
cv = KFold(10, shuffle=True, random_state=random_state)

for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
    print(target_var)
    X, y = create_data(target=target_var, feature_data=netmats_pearson)

    predicted_discovery_cv = np.zeros_like(y)
    cor_per_fold = np.zeros(cv.n_splits)
    i = 0
    for train, test in cv.split(X=X, y=y):
        model.fit(X=X[train], y=y[train])
        predicted_discovery_cv[test] = model.predict(X=X[test])
        cor_per_fold[i] = np.corrcoef(y[test], predicted_discovery_cv[test])[0,1]
        i += 1
    # correlation between the cross-validated predictions and observations in the discovery sample
    # this is the correct, unbiased estimate!
    # calculated as mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # finalize model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X, y=y)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)

    print('r =', np.round(r_disc_cv, 2), '\tp =', np.round(p_disc_cv, 3), '\tR2 =', np.round(r_disc_cv**2 * 100, 1),  '%')

age
r = 0.45 	p = 0.001 	R2 = 20.0 %
CogTotalComp_AgeAdj
r = 0.4 	p = 0.001 	R2 = 16.2 %
PMAT24_A_CR
r = 0.25 	p = 0.001 	R2 = 6.3 %
Flanker_AgeAdj
r = 0.16 	p = 0.001 	R2 = 2.6 %
CardSort_AgeAdj
r = 0.17 	p = 0.001 	R2 = 2.8 %
PicSeq_AgeAdj
r = 0.23 	p = 0.001 	R2 = 5.5 %


With Ridge and partial correlation:

In [15]:
model = Ridge()
random_state = 42
cv = KFold(10, shuffle=True, random_state=random_state)

bar_data_ridge = []

for target_var in ['age', 'CogTotalComp_AgeAdj', 'PMAT24_A_CR', 'Flanker_AgeAdj', 'CardSort_AgeAdj', 'PicSeq_AgeAdj']:
    print(target_var)
    X, y = create_data(target=target_var, feature_data=netmats_parcor)

    predicted_discovery_cv = np.zeros_like(y)
    cor_per_fold = np.zeros(cv.n_splits)
    i = 0
    for train, test in cv.split(X=X, y=y):
        model.fit(X=X[train], y=y[train])
        predicted_discovery_cv[test] = model.predict(X=X[test])
        cor_per_fold[i] = np.corrcoef(y[test], predicted_discovery_cv[test])[0,1]
        i += 1
    # correlation between the cross-validated predictions and observations in the discovery sample
    # this is the correct, unbiased estimate!
    # calculated as mean test performance across all folds
    r_disc_cv = np.mean(cor_per_fold)
    # finalize model by training it on the full discovery sample (without cross-validation)
    final_model = model.fit(X=X, y=y)
    # obtain predictions with the final model on the discovery sample, note that this model actually overfits this sample.
    # we do this only to demonstrate biased estimates
    predicted_discovery_overfit = final_model.predict(X=X)
    # here we obtain the biased effect size (r) estimates for demonstrational purposes
    r_disc_overfit = np.corrcoef(predicted_discovery_overfit, y)[0, 1]

    # below we calculate permutation-based p-values for all three effect size estimates (in-sample unbiased, in-sample biased, out-of-sample)
    # (one sided tests, testing for positive correlation)
    p_disc_cv = permutation_test(predicted_discovery_cv, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)
    p_disc_overfit = permutation_test(predicted_discovery_overfit, y, method='approximate', num_rounds=1000, func=lambda x, y: np.corrcoef(x, y)[1][0],seed=random_state)

    bar_data_ridge.append(r_disc_cv)

    print('r =', np.round(r_disc_cv, 2), '\tp =', np.round(p_disc_cv, 3), '\tR2 =', np.round(r_disc_cv**2 * 100, 1),  '%')

age
r = 0.52 	p = 0.001 	R2 = 26.7 %
CogTotalComp_AgeAdj
r = 0.5 	p = 0.001 	R2 = 25.0 %
PMAT24_A_CR
r = 0.28 	p = 0.001 	R2 = 8.1 %
Flanker_AgeAdj
r = 0.15 	p = 0.001 	R2 = 2.1 %
CardSort_AgeAdj
r = 0.24 	p = 0.001 	R2 = 5.8 %
PicSeq_AgeAdj
r = 0.17 	p = 0.001 	R2 = 2.8 %


### *See the notebook called 'plot_results_FC.ipynb' for figures.*